## Objectives
- Raw tables(ex. levelup, login, death)를 읽어오고, config.py에서 정의한 simple_preprocess 함수를 통해 간단히 전처리한다.
- Sample selection을 실시한다.
- 캐릭터별로 레벨업한 트랜잭션이 기록된 panel data를 생성한다.
- 이후 이어지는 노트북들에서는 본 노트북에서 생성된 panel data에 feature를 추가해나간다.

In [1]:
from config import *

In [2]:
# levelup history
levelups = pd.read_csv(data_dir+"characters_levelup.csv", encoding="utf-8")
levelups.drop(columns=["id","player_id","character_id","event_timestamp"], inplace=True)
levelups = simple_preprocess(levelups)
levelups = levelups.sort_values(by=["event_datetime","level_from"]).reset_index(drop=True)

In [3]:
# login history: player
logins_p = pd.read_csv(data_dir+"players_login.csv", encoding="utf-8")
logins_p = logins_p[["player_slug","event_datetime"]]
logins_p = simple_preprocess(logins_p)

In [4]:
# login history: character
logins_c = pd.read_csv(data_dir+"characters_login.csv", encoding="utf-8")
logins_c.drop(columns=["id","player_id","character_id","event_timestamp"], inplace=True)
logins_c = simple_preprocess(logins_c)

In [5]:
# death history
deaths = pd.read_csv(data_dir+"main-deaths.csv", encoding="utf-8")
deaths = simple_preprocess(deaths)

FileNotFoundError: [Errno 2] No such file or directory: '../data_raw/main-deaths.csv'

In [6]:
# ads watching history
ads = pd.read_csv(data_dir+"characters_ads.csv", encoding="utf-8").drop(columns=["id"])
ads = ads.query("ad_status=='start'") # select started transactions only
ads = simple_preprocess(ads)

In [7]:
# purchase history
purchases = pd.read_csv(data_dir+"purchases.csv", encoding="utf-8")
purchases.drop(columns=["id","purchase_quantity","purchase_unit_price","purchase_currency_code"], inplace=True)
purchases = simple_preprocess(purchases)

# Sample Players

In [8]:
# list of sample players
players = logins_p[["player_slug"]].drop_duplicates().sort_values(by="player_slug").reset_index(drop=True)

# the day when each player registered
players["dt_player_register"] = players["player_slug"].apply(lambda x: x[:-5].replace("T"," "))
players["dt_player_register"] = pd.to_datetime(players["dt_player_register"]) + datetime.timedelta(hours=9) # UTC to KST

len(players)

148189

* Players who registered between November 1, 2021 and March 31, 2022

In [9]:
a = len(players)
players = players.query("@window_sday<=dt_player_register<=@window_eday").reset_index(drop=True)
b = len(players)
print(a - b, "dropped")
print(b)

2272 dropped
145917


* Players who created only one character

In [10]:
a = len(players)

tmp = levelups.groupby("player_slug")["character_slug"].nunique()
tmp = tmp[tmp==1].index
players = players.query("player_slug in @tmp").reset_index(drop=True)

tmp = logins_c.groupby("player_slug")["character_slug"].nunique()
tmp = tmp[tmp==1].index
players = players.query("player_slug in @tmp").reset_index(drop=True)

b = len(players)
print(a - b, "dropped")
print(b)

82686 dropped
63231


* Players whose level 1 is observed

In [11]:
a = len(players)

# players whose level up from 1 to 2 is not observable
tmp = levelups.groupby(["player_slug","character_slug"])["level_from"].min()
tmp = tmp[tmp==1].reset_index()["player_slug"].unique()
players = players.query("player_slug in @tmp").reset_index(drop=True)

tmp = levelups.groupby(["player_slug","character_slug"])["level_from"].min()
tmp = tmp[tmp!=1].reset_index()["player_slug"].unique()
players = players.query("player_slug not in @tmp").reset_index(drop=True)

b = len(players)
print(a - b, "dropped")
print(b)

639 dropped
62592


In [12]:
a = len(players)

In [13]:
# players who have duplicated levelup transactions with the same character name
# 1) recording error
# 2) players who actually created at least two characters with the same name
tmp = levelups.groupby(["player_slug","character_slug","level_from"]).size()
tmp = tmp[tmp>1].reset_index().player_slug.unique()
players = players.query("player_slug not in @tmp").reset_index(drop=True)

In [14]:
# 레벨업 1->2 duration 음수 나오거나 레벨업 시간 missing 있는 플레이어들
tmp = ['2021-12-18T07:37:52.929Z','2021-12-19T05:01:11.551Z','2021-12-19T23:00:06.232Z','2021-12-17T12:46:39.821Z',
       '2021-12-18T03:13:36.612Z','2021-12-23T07:53:20.294Z','2021-12-28T08:03:59.202Z','2021-12-31T16:05:54.060Z',
       '2021-12-18T17:16:55.915Z','2021-12-24T14:01:23.800Z','2021-12-21T03:33:59.462Z',
       
       '2021-12-24T00:02:08.718Z','2021-12-23T02:21:04.360Z',
       '2021-12-18T14:09:36.460Z','2021-12-24T13:41:09.863Z','2021-12-21T05:23:36.954Z']
players = players.query("player_slug not in @tmp").reset_index(drop=True)

In [15]:
# players whose all level ups are observed
tmp = levelups.groupby(["player_slug","character_slug"]).size().to_frame("n_obs").reset_index()
tmp_ = levelups.groupby(["player_slug","character_slug"]).level_from.min().to_frame("lv_min").reset_index()
tmp = pd.merge(tmp, tmp_)
tmp_ = levelups.groupby(["player_slug","character_slug"]).level_to.max().to_frame("lv_max").reset_index()
tmp = pd.merge(tmp, tmp_)
tmp = tmp.query("lv_max-lv_min != n_obs").player_slug.unique()
players = players.query("player_slug not in @tmp").reset_index(drop=True)

In [16]:
b = len(players)
print(a - b, "dropped")
print(b)

3250 dropped
59342


In [17]:
players.to_csv("../data/sample_players.csv", index=None, encoding="utf-8")

* Apply the sample restriction to other tables

In [18]:
levelups = levelups.query("player_slug in @players.player_slug.unique()").reset_index(drop=True)
logins_p = logins_p.query("player_slug in @players.player_slug.unique()").reset_index(drop=True)
logins_c = logins_c.query("player_slug in @players.player_slug.unique()").reset_index(drop=True)
deaths = deaths.query("player_slug in @players.player_slug.unique()").reset_index(drop=True)
ads = ads.query("player_slug in @players.player_slug.unique()").reset_index(drop=True)
purchases = purchases.query("player_slug in @players.player_slug.unique()").reset_index(drop=True)

# Character-Character Level-Level Panel

In [19]:
levelups_ref = levelups.copy()

* <code>death</code>: Total number of deaths occurred at each level

In [20]:
deaths_ref = pd.merge_asof(deaths, levelups[["player_slug","character_slug","level_to","event_datetime"]],
                           by=["player_slug","character_slug"], on="event_datetime", direction="backward").reset_index(drop=True)

# character level when each death was made
deaths_ref["level_to"] = deaths_ref["level_to"].fillna(1).astype(int)
deaths_ref.rename(columns={"level_to":"level_from"}, inplace=True)

# total deaths made at each character level
tmp = deaths_ref.groupby(["player_slug","character_slug","level_from"]).size()
levelups_ref = pd.merge(levelups_ref, tmp.to_frame("death").reset_index(), how="left")
levelups_ref["death"].fillna(0, inplace=True)

* <code>ad_revival</code>: Total number of ads for revival watched at each level
* <code>ad_item</code>: Total number of ads for items watched at each level

In [21]:
# ads for revival
ads_ref = pd.merge_asof(ads.query("ad_slug=='Revival'"),
                        levelups[["player_slug","character_slug","level_to","event_datetime"]],
                        by=["player_slug","character_slug"], on="event_datetime", direction="backward").reset_index(drop=True)

# character level when each ad was watched
ads_ref["level_to"] = ads_ref["level_to"].fillna(1).astype(int)
ads_ref.rename(columns={"level_to":"level_from"}, inplace=True)

# number of ads watched at each character level
tmp = ads_ref.groupby(["player_slug","character_slug","level_from"]).size()
levelups_ref = pd.merge(levelups_ref, tmp.to_frame("ad_revival").reset_index(), how="left")
levelups_ref["ad_revival"].fillna(0, inplace=True)

In [22]:
# ads for items
ads_ref = pd.merge_asof(ads.query("ad_slug!='Revival'"),
                        levelups[["player_slug","character_slug","level_to","event_datetime"]],
                        by=["player_slug","character_slug"], on="event_datetime", direction="backward").reset_index(drop=True)

# character level when each ad was watched
ads_ref["level_to"] = ads_ref["level_to"].fillna(1).astype(int)
ads_ref.rename(columns={"level_to":"level_from"}, inplace=True)

# number of ads watched at each character level
tmp = ads_ref.groupby(["player_slug","character_slug","level_from"]).size()
levelups_ref = pd.merge(levelups_ref, tmp.to_frame("ad_item").reset_index(), how="left")
levelups_ref["ad_item"].fillna(0, inplace=True)

In [23]:
levelups_ref["ad_total"] = levelups_ref[["ad_revival","ad_item"]].sum(axis=1)

* <code>purchase</code>: Total purchases made at each level

In [24]:
purchases_ref = pd.merge_asof(purchases, levelups[["player_slug","character_slug","level_to","event_datetime"]],
                              by=["player_slug","character_slug"], on="event_datetime", direction="backward").reset_index(drop=True)

# character level when each purchase was made
purchases_ref["level_to"] = purchases_ref["level_to"].fillna(1).astype(int)
purchases_ref.rename(columns={"level_to":"level_from"}, inplace=True)

# total purchases made at each character level
tmp = purchases_ref.groupby(["player_slug","character_slug","level_from"])["purchase_total_price"].sum()
levelups_ref = pd.merge(levelups_ref, tmp.to_frame("purchase").reset_index(), how="left")
levelups_ref["purchase"].fillna(0, inplace=True)

* <code>interval</code>: Hours taken to progress to the next level

In [25]:
# previous levelups datetime
levelups_ref["event_datetime_prev"] = levelups_ref.groupby(["player_slug","character_slug"])["event_datetime"].shift(1)

# when level_from is 1 and level_to is 2
tmp = logins_c.groupby(["player_slug","character_slug"])["event_datetime"].min().to_frame("first_login_datetime").reset_index()
levelups_ref = pd.merge(levelups_ref, tmp, how="left")
levelups_ref["event_datetime_prev"] = np.where(levelups_ref["event_datetime_prev"]==levelups_ref["event_datetime_prev"],
                                               levelups_ref["event_datetime_prev"], levelups_ref["first_login_datetime"])

# calculate interval
levelups_ref["interval"] = levelups_ref["event_datetime"] - levelups_ref["event_datetime_prev"]
levelups_ref["interval"] = levelups_ref["interval"].apply(lambda x: (x.total_seconds()+1)/3600)
levelups_ref.drop(columns=["first_login_datetime"], inplace=True)

levelups_ref.query("interval<0 | interval!=interval").player_slug.unique() # error check

array([], dtype=object)

In [26]:
levelups_ref[["interval"]].describe()

,interval
count,2.466997e+06
mean,1.110537e+00
std,2.624410e+01
min,2.777778e-04
25%,1.000000e-02
50%,2.361111e-02
75%,4.305556e-02
max,3.164869e+03


In [27]:
# cumulative sum of intervals
levelups_ref["interval_cum"] = levelups_ref.groupby(["player_slug","character_slug"])["interval"].cumsum()

* Export

In [28]:
levelups_ref.player_slug.nunique()

59342

In [29]:
levelups_ref = levelups_ref.sort_values(by=["event_datetime","level_from"]).reset_index(drop=True) # several leveups can happen at once
levelups_ref

,player_slug,character_slug,level_from,level_to,event_datetime,death,ad_revival,ad_item,ad_total,purchase,event_datetime_prev,interval,interval_cum
0,2021-11-01T07:31:49.366Z,레드스완,1,2,2021-11-01 16:33:55,0.0,0.0,0.0,0.0,0.0,2021-11-01 16:33:19,0.010278,0.010278
1,2021-11-01T07:31:49.366Z,레드스완,2,3,2021-11-01 16:34:12,0.0,0.0,0.0,0.0,0.0,2021-11-01 16:33:55,0.005000,0.015278
2,2021-11-01T07:31:49.366Z,레드스완,3,4,2021-11-01 16:36:50,0.0,0.0,0.0,0.0,0.0,2021-11-01 16:34:12,0.044167,0.059444
3,2021-11-01T07:31:49.366Z,레드스완,4,5,2021-11-01 16:38:30,0.0,0.0,0.0,0.0,0.0,2021-11-01 16:36:50,0.028056,0.087500
4,2021-11-01T07:31:49.366Z,레드스완,5,6,2021-11-01 16:39:43,0.0,0.0,0.0,0.0,0.0,2021-11-01 16:38:30,0.020556,0.108056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466992,2022-03-30T14:13:23.546Z,sheal,19,20,2022-03-30 23:56:33,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:55:16,0.021667,0.705556
2466993,2022-03-30T06:19:53.643Z,반백살,36,37,2022-03-30 23:56:47,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:46:38,0.169444,8.593889
2466994,2022-03-30T14:13:23.546Z,sheal,20,21,2022-03-30 23:57:47,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:56:33,0.020833,0.726389
2466995,2022-03-30T14:13:23.546Z,sheal,21,22,2022-03-30 23:58:07,0.0,0.0,0.0,0.0,0.0,2022-03-30 23:57:47,0.005833,0.732222


In [30]:
levelups_ref.to_csv("../data/levelups_panel_1.csv", index=None, encoding="utf-8")